# STEP 1: CONNECT TO SQL [POWER PLANT]

In [ ]:
# CONNECT TO SQL

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, declarative_base

# 1. Define the database file location
# "sqlite:///" tells SQLAlchemy to use SQLite.
# "bookings.db" is the file that will be created in this folder.
DATABASE_URL = "sqlite:///bookings.db"

# 2. Create the Engine
# The "connect_args" part is specific to SQLite to allow
# one thread to talk to the DB while another waits (prevents locking errors).
engine = create_engine(
    DATABASE_URL, 
    connect_args={"check_same_thread": False}
)

# 3. Create the Session Factory
# Think of this as a "dispenser" for database sessions.
# When we need to read/write, we will ask SessionLocal for a session.
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# 4. Create the Base Class
# All our models (User, Slot, Booking) will inherit from this "Base".
# It allows SQLAlchemy to track them.
Base = declarative_base()

# STEP 2: DATABASE STRUCTURE
## __________ DEFINES TABLES (USERS, SLOT, BOOKING)

In [ ]:
# STEP 2: DATABASE STRUCTURE

from sqlalchemy import Column, Integer, String, DateTime, Boolean, ForeignKey
from sqlalchemy.orm import relationship
from datetime import datetime
from database import Base

# 1. The User Table
class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True, index=True)
    username = Column(String, nullable=False)
    email = Column(String, unique=True, index=True, nullable=False)
    role = Column(String, default="student")  # 'student' or 'volunteer'
    created_at = Column(DateTime, default=datetime.utcnow)

    # Relationships
    # "slots" allows us to do user.slots to see what they volunteered for
    slots = relationship("Slot", back_populates="volunteer")
    # "bookings" allows us to do user.bookings to see what they have booked
    bookings = relationship("Booking", back_populates="student")

# 2. The Slot Table (The "Supply")
class Slot(Base):
    __tablename__ = 'slots'

    id = Column(Integer, primary_key=True, index=True)
    volunteer_id = Column(Integer, ForeignKey('users.id'))
    start_time = Column(DateTime, nullable=False)
    end_time = Column(DateTime, nullable=False)
    is_booked = Column(Boolean, default=False)

    # Relationships
    volunteer = relationship("User", back_populates="slots")
    booking = relationship("Booking", back_populates="slot", uselist=False)

# 3. The Booking Table (The "Demand")
class Booking(Base):
    __tablename__ = 'bookings'

    id = Column(Integer, primary_key=True, index=True)
    slot_id = Column(Integer, ForeignKey('slots.id'), unique=True)
    student_id = Column(Integer, ForeignKey('users.id'))
    google_event_id = Column(String, nullable=True) # Stores the ID from Google Calendar
    booked_at = Column(DateTime, default=datetime.utcnow)

    # Relationships
    slot = relationship("Slot", back_populates="booking")
    student = relationship("User", back_populates="bookings")

# STEP 3: DATA SETUP | USER FUNCTIONS (CRUD)

In [ ]:
from datetime import datetime
from sqlalchemy.orm import Session
from database import SessionLocal, engine, Base
from models import User, Slot, Booking

# --- DATABASE SETUP ---
def init_db():
    """Creates the database tables."""
    print("Creating database tables...")
    Base.metadata.create_all(bind=engine)
    print("Tables created successfully!")

# --- UTILITY: GET SESSION ---
def get_session():
    """
    Helper to get a database session. 
    Always use this with a context manager (with get_session() as db:)
    """
    return SessionLocal()

# --- 1. USER FUNCTIONS ---
def add_user(username: str, email: str, role: str = "student"):
    """
    Adds a new user to the database.
    """
    db = get_session()
    try:
        # Check if email exists first to avoid crashing
        existing_user = db.query(User).filter(User.email == email).first()
        if existing_user:
            print(f"Error: User with email {email} already exists.")
            return None

        new_user = User(username=username, email=email, role=role)
        db.add(new_user)
        db.commit()      # Save changes
        db.refresh(new_user) # Reload so we get the new ID
        print(f"User '{username}' created with ID: {new_user.id}")
        return new_user
    except Exception as e:
        print(f"An error occurred: {e}")
        db.rollback() # Undo changes if something broke
    finally:
        db.close()    # Always close the connection!

def get_user_by_email(email: str):
    db = get_session()
    user = db.query(User).filter(User.email == email).first()
    db.close()
    return user

# --- 2. SLOT FUNCTIONS ---
def add_slot(volunteer_email: str, start_time: datetime, end_time: datetime):
    """
    Adds a new availability slot.
    """
    db = get_session()
    try:
        # 1. Find the volunteer
        volunteer = db.query(User).filter(User.email == volunteer_email).first()
        if not volunteer:
            print(f"Error: Volunteer with email {volunteer_email} not found.")
            return None

        # 2. Create the slot
        new_slot = Slot(
            volunteer_id=volunteer.id,
            start_time=start_time,
            end_time=end_time,
            is_booked=False
        )
        db.add(new_slot)
        db.commit()
        print(f"Slot created for {volunteer.username} from {start_time} to {end_time}")
        return new_slot
    except Exception as e:
        print(f"Error creating slot: {e}")
        db.rollback()
    finally:
        db.close()

if __name__ == "__main__":
    # If we run this file directly, it initializes the DB
    init_db()